In [107]:
%reload_ext autoreload
%autoreload 2
import logging
import grpc
import pandas as pd
import sys
sys.path.append('proto')
from proto import contest_pb2
from proto import contest_pb2_grpc
from proto import question_pb2
from proto import question_pb2_grpc
import numpy as np
import time
from utils import *
from strategy_utils import *

In [108]:
contest_channel=grpc.insecure_channel('47.103.23.116: 56702')
question_channel=grpc.insecure_channel('47.103.23.116: 56701')

contest_stub=contest_pb2_grpc.ContestStub(contest_channel)
question_stub=question_pb2_grpc.QuestionStub(question_channel)

login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

session_key=login_response.session_key
init_capital=login_response.init_capital

In [4]:
df

,day,stock,open,high,low,close,volume
0,-1100.0,1000.0,19.96632,19.96632,19.06278,19.18884,299216.0
1,-1100.0,1001.0,33.59400,33.59400,33.59400,33.59400,0.0
2,-1100.0,1002.0,84.11766,84.76446,81.01170,84.50574,233872.0
3,-1100.0,1003.0,5.54400,5.54400,5.27340,5.27340,19200.0
4,-1100.0,1004.0,198.46200,203.41200,195.69000,203.41200,6736.0
...,...,...,...,...,...,...,...
386095,-1.0,1346.0,46.26732,48.90600,46.26732,48.90600,4544.0
386096,-1.0,1347.0,68.50140,69.43200,68.50140,69.43200,2400.0
386097,-1.0,1348.0,238.55700,241.23660,238.26000,238.49100,6800.0
386098,-1.0,1349.0,139.30620,141.66240,139.30620,141.17400,2288.0


In [11]:
df = pd.read_csv('./data/saved_data.csv',dtype=float).set_index(['day','stock'])
o = df['open'].unstack(level=1).values
h = df['high'].unstack(level=1).values
l = df['low'].unstack(level=1).values
c = df['close'].unstack(level=1).values
v = df['volume'].unstack(level=1).values
r = df['close'].unstack(level=1).pct_change().values

dailyfactors = get_factors_with_ohlcv(o,h,l,c,v)

In [42]:
history_weights = np.empty((0,daily_factor_weights.shape[0],daily_factor_weights.shape[1]))
for j in range(52,0,-1):
    dailyfactors = get_factors_with_ohlcv(o[-j-252:-j],h[-j-252:-j],l[-j-252:-j],c[-j-252:-j],v[-j-252:-j])
    daily_factor_weights = get_all_weights(dailyfactors,head_n=10,tail_n=10)
    history_weights = np.vstack([history_weights,daily_factor_weights.reshape((1,daily_factor_weights.shape[0],daily_factor_weights.shape[1]))])
dailyfactors = get_factors_with_ohlcv(o[-252:],h[-252:],l[-252:],c[-252:],v[-252:])
daily_factor_weights = get_all_weights(dailyfactors,head_n=10,tail_n=10)
history_weights = np.vstack([history_weights,daily_factor_weights.reshape((1,daily_factor_weights.shape[0],daily_factor_weights.shape[1]))])

In [61]:
new_hr = np.dot(history_weights[i-52],r[i-50])

In [105]:
init_weight = np.ones(daily_factor_weights.shape[0]) / daily_factor_weights.shape[0]
hr = np.empty((0,daily_factor_weights.shape[0]))
for i in range(50):
    factor_weights,hr = update_weights_history(history_weights[i-52],r[i-50],hr)

In [112]:
i=0#控制seq
count=0
data_lst=df.values.tolist()
# factors_lst=[]
period=1#eg 每两天跑一次策略
comission=0#
max_exposure=0.1#大盘上涨，多头增加，大盘下跌，空头增加
single_stock_position_limit=0.1#
lending_rate=0.01#
borrow_rate=0.05#
leverage=2#
# factors=pd.DataFrame()
prev_factors=pd.DataFrame()#上一次计算的factors
# prepare history data
# o 
# h 
# l 
# c 
# v 
# p 
# history_weights = np.ndarray()

In [10]:
# history_weights = np.empty(0)

In [118]:
while True:
    login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

    session_key=login_response.session_key
    init_capital=login_response.init_capital
    i = 0
    # try:
    while True:
        question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88,sequence=i))
        print(question_response.sequence)
        if question_response.sequence!=-1:
            if i == 0 :
                i = question_response.sequence
            dailystk = [x.values for x in question_response.dailystk]

            daily = np.asarray(dailystk)
            o = np.vstack([o,daily[:,2]])
            h = np.vstack([o,daily[:,3]])
            l = np.vstack([o,daily[:,4]])
            c = np.vstack([o,daily[:,5]])
            v = np.vstack([o,daily[:,6]])
            if count>-1:#开始不动，只要有新数据就跑一次策略
                print('run strategy')

                
                dailyfactors=get_factors(df,prev_factors)  #从数据获取因子

                dailyfactors = get_factors_with_ohlcv(o,h,l,c,v)
                # prev_factors=dailyfactors

                daily_factor_weights = get_all_weights(dailyfactors,head_n=10,tail_n=10) # shape: 351 * factors#

                daily_trade_weights = get_trade_weights(daily_factor_weights,factor_weights,bias=1) # shape: 351 * 1


                # factors_lst.extend(dailyfactors.values)#向因子库追加
                #部份因子可能需要根据历史因子数据归一化

                # factor_select=select_factors(factors,n=10,period=period)  #计算相关系数选取因子
                # factor_select=['avg', 'mom', 'max52', 'cci', 'K', 'D', 'J', 'rsi', 'trix', 'willr']
                # factor_select=['avg','mom', 'vol', 'max52', 'K', 'D', 'J', 'rsi', 'trix', 'willr', 'macd', 'natr','mfi']
                # factor_select=['avg', 'mom', 'max52', 'D', 'willr', 'natr', 'rsi']

                # index_direction='neutral'#TODO 大盘方向，用于控制exposure

                # weights = get_weight(dailyfactors[factor_select],head_n=10,tail_n=10)
                if count < 100:
                    target_pos=get_position(daily_trade_weights,
                                            pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                            question_response.positions,
                                            question_response.capital / 2,
                                            comission)
                else:
                    target_pos=get_position(daily_trade_weights,
                                            pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                            question_response.positions,
                                            question_response.capital,
                                            comission)
                
                #提交策略
                if count%period==0:#按周期提交
                    ##summit answer
                    submit_response = contest_stub.submit_answer(contest_pb2.AnswerRequest(user_id=88,user_pin='dDTSvdwk',session_key=login_response.session_key,sequence=i,positions=target_pos))

                    print(submit_response,question_response.capital)
                    if not submit_response.accepted:
                        print(submit_response.reason)
                        if submit_response.reason[-7:] == 'timeout':
                            i=question_response.sequence+1
                            count+=1
                            continue#如果提交超时，直接请求新数据
                history_weights = np.vstack([history_weights,daily_factor_weights.reshape((1,daily_factor_weights.shape[0],daily_factor_weights.shape[1]))])
                pctchg = c[-1,:] / c[-2,:] - 1
                r = np.vstack([r,pctchg])
                factor_weights,hr = update_weights_history(history_weights[-2],pctchg,hr)
                data_lst.extend(dailystk)
                df=pd.DataFrame(data_lst,columns=['day','stock','open','high','low','close','volume'], dtype=float).set_index(['day','stock'])
                df.to_csv('./data/saved_data.csv')
            i=question_response.sequence+1
            count+=1
        time.sleep(1)
            # if count==20:
                # break
    # except Exception as e:
    #     print(e)
    time.sleep(1)

502
run strategy
stock
1016.0       53169
1023.0      397156
1059.0      163736
1060.0      471017
1065.0     2618430
1069.0     2847488
1080.0      552958
1086.0      504241
1087.0     1243556
1091.0     3257840
1158.0      370006
1160.0      258167
1176.0      479603
1188.0     1311571
1192.0      153943
1196.0      187604
1205.0    12222156
1210.0     2366563
1218.0     1316127
1225.0      157869
1232.0     1353167
1267.0     3898012
1276.0       48189
1277.0      635716
1282.0       98874
1295.0       88405
1296.0      403156
1307.0       74985
1312.0       79258
1314.0      143626
1315.0      859543
1317.0      558336
1329.0      203658
1346.0     1058934
Name: close, dtype: int32
reason: "exceed max exposure, long value=537639328.81 and short_value=0.00"
 537640443.093891
exceed max exposure, long value=537639328.81 and short_value=0.00


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 14 and the array at index 1 has size 351

In [116]:
daily_factor_weights.shape

(14, 351)

In [120]:
history_weights

array([-0., -0., -0., ..., -0., -0., -0.])